In [13]:
import csv
import numpy as np
import nltk
from nltk import word_tokenize

In [2]:
csvfile = open('question_train.csv', 'r')
reader = csv.reader(csvfile, delimiter=',', quotechar='"', lineterminator='<CR>')

In [3]:
csvfile.seek(0);
#cols = list(zip(np.arange(21),next(reader)))
for i, q in zip(np.arange(21), next(reader)):
    print('{0}: {1}'.format(i,q))

0: question_id
1: user_id
2: sms_guru_id
3: category_main_id
4: question
5: description
6: tags
7: categories
8: url
9: rating_count_positive
10: rating_count_negative
11: answer_count
12: reported
13: answered
14: active
15: deleted
16: seo_locked
17: editor_locked
18: editor_id
19: created_at
20: updated_at


# checking for ill-shaped lines

In [4]:
csvfile.seek(0);
i = 0 #count total row number
j = 0 #count false row number

for row in reader:
    i += 1
    if(len(row) != 21):
        #print('line size error in line {} \n'.format(reader.line_num))
        #print('line size is: {} \n'.format(len(row)))
        #print(*row)
        #print('\n')
        j += 1

print('Total number of rows: {0} \n Number of rows with wrong length: {1}:\n fraction: {2:.2f} %'.format(i,j,(j/i)*100))

Total number of rows: 16070 
 Number of rows with wrong length: 57:
 fraction: 0.35 %


# Reading out questions and tokenizing, checking vocabulary

In [51]:
csvfile.seek(0);
next(reader)

questions = []
vocab = set([])

for row in reader:
    if(len(row) == 21):
        words = word_tokenize(row[4])
        questions.append( (row[0], words) )
        vocab = vocab.union( words )

# Checking Vocabulary

In [70]:
totNum = sum([len(q[1]) for q in questions])
vocNum = len(vocab)
print("""The total number of words is {0}. 
The volume of the vocabulary is {1}.
That makes an percentiage of {2:.2}""".format(totNum,vocNum,vocNum/totNum))

The total number of words is 140439. 
The volume of the vocabulary is 23603.
That makes an percentiage of 0.17


In [88]:
allwordlist = [w for q in questions for w in q[1]]
fd = nltk.FreqDist(allwordlist)

In [109]:
for d in fd.most_common(20):
    print("{0:10} x {1:6}".format(*d) )

?          x  11576
der        x   4572
Wie        x   3121
ist        x   3114
Was        x   3035
die        x   2270
und        x   2014
ich        x   1738
in         x   1584
bedeutet   x   1439
das        x   1322
hat        x   1199
von        x   1185
.          x   1161
name       x   1134
ein        x   1089
es         x   1077
,          x   1061
man        x   1056
kommt      x    895


In [77]:
vocabLis = sorted(list(vocab))
vocabLis[-300:-200]

['zinkblende',
 'zinsen',
 'zionistin',
 'zirkus',
 'zita',
 'zitat',
 'zivildienstgesetz',
 'zocke',
 'zoey',
 'zoey101',
 'zoff',
 'zog',
 'zoi',
 'zoj',
 'zoll',
 'zoloft',
 'zombis',
 'zone',
 'zonen',
 'zonk',
 'zorro',
 'zs',
 'zsdlj',
 'zu',
 'zuammen',
 'zucchini',
 'zuchini',
 'zuckmayer',
 'zueinander',
 'zuerst',
 'zufahren',
 'zuficken',
 'zufällig',
 'zufüllen',
 'zug',
 'zugabe',
 'zugegeben',
 'zugelassen',
 'zugelassene',
 'zugeschlossene',
 'zugleich',
 'zugreifen',
 'zugunglück',
 'zugverbindung',
 'zugängliche',
 'zuhaus',
 'zuhause',
 'zuhälter',
 'zukleben',
 'zukommen',
 'zukunft',
 'zukünftige',
 'zukünftigen',
 'zukünftiger',
 'zulassen',
 'zulassung',
 'zulassungsdienst',
 'zulässg',
 'zulässiges',
 'zum',
 'zumachen',
 'zumauern',
 'zumindest',
 'zunge',
 'zungenkrebs',
 'zungenkuss',
 'zungenpircing',
 'zungkuss',
 'zupfmassagen',
 'zur',
 'zurck',
 'zurecht',
 'zurich',
 'zurufen',
 'zurzeit',
 'zurück',
 'zurück.Ich',
 'zurückblickt',
 'zurückgekehrten',
 '

# Playing area

In [125]:
import re
for q in questions[:10]:
    print([w for w in q[1] if re.search('^W',w)])

['Was']
['WIEVIEL']
['Was']
[]
[]
['Wie']
['Wie']
['Wie']
[]
['Was']


In [21]:
for x in range(1,11):
    print(repr(x).zfill(5), repr(x**2).ljust(5),repr(x**3).ljust(5))

00001 1     1    
00002 4     8    
00003 9     27   
00004 16    64   
00005 25    125  
00006 36    216  
00007 49    343  
00008 64    512  
00009 81    729  
00010 100   1000 


In [13]:
a = np.arange(5)
b = np.arange(5) +5

In [18]:
list(zip(a,b))

[(0, 5), (1, 6), (2, 7), (3, 8), (4, 9)]

In [108]:
print('Hällülöliös\
    ')

Hällülöliös    


In [109]:
nacute = '\u0144'

In [110]:
nacute

'ń'